# 🧹 Phase 1 – Data Cleaning & EDA
**Objective:**  
Clean and preprocess employee and project datasets to ensure data quality and readiness for feature engineering.

---

## ✅ Steps:
1. Import Libraries & Load Data
2. Initial Data Exploration
3. Data Cleaning
4. Skill & Certification Normalization
5. Save Cleaned Datasets


In [42]:
# ================================================
# 1. Import Required Libraries
# ================================================
import pandas as pd
import numpy as np
import re

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

# File paths (adjust as needed)
EMPLOYEE_MASTER_PATH = "../data/raw/employee_master.csv"
EMPLOYEE_EXP_PATH = "../data/raw/employee_experience.csv"
CLIENT_PROJECTS_PATH = "../data/raw/client_projects.csv"


In [43]:
# ================================================
# 2. Load Raw Datasets
# ================================================
df_emp = pd.read_csv(EMPLOYEE_MASTER_PATH)
df_exp = pd.read_csv(EMPLOYEE_EXP_PATH)
df_proj = pd.read_csv(CLIENT_PROJECTS_PATH)

# Display dataset shapes
print("Employee Master:", df_emp.shape)
print("Employee Experience:", df_exp.shape)
print("Client Projects:", df_proj.shape)

# Preview datasets
display(df_emp.head())
display(df_exp.head())
display(df_proj.head())


Employee Master: (1000, 6)
Employee Experience: (1000, 2)
Client Projects: (100, 6)


,Employee_ID,Name,Department,Years_Experience,Current_Project_ID,Location
0,E101,Mariah Brennan,AI Research,14.5,NaN,Pune
1,E102,Thomas Terry,DevOps,2.2,NaN,Pune
2,E103,Vanessa Jackson MD,Full Stack Dev,3.9,NaN,Mumbai
3,E104,Lori Sanchez,AI Research,8.3,NaN,Chennai
4,E105,Nicole Warner,Data Science,6.0,NaN,Mumbai


,Employee_ID,Experience_Text
0,E101,"Worked on projects involving Azure, Blockchain..."
1,E102,"Worked on projects involving REST APIs, Data E..."
2,E103,Worked on projects involving Data Visualizatio...
3,E104,"Worked on projects involving GraphQL, Agile, a..."
4,E105,"Worked on projects involving Java, Azure, and ..."


,Project_ID,Client_Name,Project_Description,Required_Skills,Location,Status
0,P301,Byrd Ltd,Looking for expertise in SQL Database Manageme...,"SQL Database Management, Docker, NoSQL",Delhi,Closed
1,P302,"Cunningham, Anderson and Fernandez","Looking for expertise in Cloud Security, Conta...","Cloud Security, Container Orchestration, Python",Pune,Open
2,P303,"Mata, Mclean and Jones","Looking for expertise in React, Mobile Develop...","React, Mobile Development, Microservices Archi...",Mumbai,Closed
3,P304,Rodriguez-Dominguez,"Looking for expertise in Edge Computing, Jenki...","Edge Computing, Jenkins, IoT",Chennai,Closed
4,P305,Medina-Maldonado,"Looking for expertise in Terraform, SQL Databa...","Terraform, SQL Database Management, NLP",Mumbai,Open


In [44]:
# ================================================
# 3. Initial EDA
# ================================================
# Check for missing values
print("\nMissing Values:")
print("Employee Master:\n", df_emp.isnull().sum())
print("Employee Experience:\n", df_exp.isnull().sum())
print("Client Projects:\n", df_proj.isnull().sum())

# Basic stats
print("\nSummary Statistics (Years of Experience):")
print(df_emp["Years_Experience"].describe())

# Unique Departments & Locations
print("\nDepartments:", df_emp["Department"].unique())
print("Locations:", df_emp["Location"].unique())



Missing Values:
Employee Master:
 Employee_ID              0
Name                     0
Department               0
Years_Experience         0
Current_Project_ID    1000
Location                 0
dtype: int64
Employee Experience:
 Employee_ID        0
Experience_Text    0
dtype: int64
Client Projects:
 Project_ID             0
Client_Name            0
Project_Description    0
Required_Skills        0
Location               0
Status                 0
dtype: int64

Summary Statistics (Years of Experience):
count    1000.000000
mean        8.130000
std         4.128511
min         1.000000
25%         4.400000
50%         8.400000
75%        11.700000
max        14.900000
Name: Years_Experience, dtype: float64

Departments: ['AI Research' 'DevOps' 'Full Stack Dev' 'Data Science' 'Cybersecurity'
 'Cloud Engineering']
Locations: ['Pune' 'Mumbai' 'Chennai' 'Delhi']


In [45]:
# ================================================
# 4. Data Cleaning & Normalization
# ================================================

# ---- Employee Master ----
df_emp["Years_Experience"] = pd.to_numeric(df_emp["Years_Experience"], errors="coerce").fillna(0)

# ---- Employee Experience ----
df_exp["Experience_Text"] = df_exp["Experience_Text"].astype(str).str.lower().str.strip()

# ---- Client Projects ----
df_proj["Project_Description"] = df_proj["Project_Description"].astype(str).str.lower().str.strip()

def normalize_skills(skill_str):
    """Normalize skills by lowering case, stripping spaces, and removing duplicates"""
    if pd.isna(skill_str):
        return ""
    skills = [s.strip().lower() for s in skill_str.split(",")]
    return ", ".join(sorted(set(skills)))

df_proj["Required_Skills"] = df_proj["Required_Skills"].apply(normalize_skills)


In [46]:
# ================================================
# 5. Remove unwanted columns
# ================================================
print(df_emp.head())  # Display the first few rows of the cleaned employee master dataset

df_emp.drop(columns=["Current_Project_ID", "Name"], inplace=True, errors='ignore')

print("\nCleaned Employee Master:")
print(df_emp.head())  # Display the first few rows after dropping unwanted columns

print(df_exp.head())

print(df_proj.head())

  Employee_ID                Name      Department  Years_Experience  Current_Project_ID Location
0        E101      Mariah Brennan     AI Research              14.5                 NaN     Pune
1        E102        Thomas Terry          DevOps               2.2                 NaN     Pune
2        E103  Vanessa Jackson MD  Full Stack Dev               3.9                 NaN   Mumbai
3        E104        Lori Sanchez     AI Research               8.3                 NaN  Chennai
4        E105       Nicole Warner    Data Science               6.0                 NaN   Mumbai

Cleaned Employee Master:
  Employee_ID      Department  Years_Experience Location
0        E101     AI Research              14.5     Pune
1        E102          DevOps               2.2     Pune
2        E103  Full Stack Dev               3.9   Mumbai
3        E104     AI Research               8.3  Chennai
4        E105    Data Science               6.0   Mumbai
  Employee_ID                                    E

In [47]:
# Replace '.net' (case-insensitive) with 'Dot Net' in Experience_Text
df_exp["Experience_Text"] = df_exp["Experience_Text"].str.replace(r'\.net', 'Dot Net', case=False, regex=True)

# Replace '.net' in Required_Skills of client projects
df_proj["Required_Skills"] = df_proj["Required_Skills"].str.replace(r'\.net', 'Dot Net', case=False, regex=True)


In [48]:
# ================================================
# 5. Save Cleaned Datasets
# ================================================
OUTPUT_EMP_PATH = "../data/processed/employee_master_cleaned.csv"
OUTPUT_EXP_PATH = "../data/processed/employee_experience_cleaned.csv"
OUTPUT_PROJ_PATH = "../data/processed/client_projects_cleaned.csv"

df_emp.to_csv(OUTPUT_EMP_PATH, index=False)
df_exp.to_csv(OUTPUT_EXP_PATH, index=False)
df_proj.to_csv(OUTPUT_PROJ_PATH, index=False)

print("✅ Cleaned datasets saved successfully to /data/processed/")


✅ Cleaned datasets saved successfully to /data/processed/
